In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import sweetviz as sv

In [8]:
path = "CoffeeEconomies.xlsx"

coffee_df = pd.read_excel(path)

legend = pd.read_excel("pwt1001.xlsx", sheet_name="Legend", header=1)

"""Dropping useless meta-variables"""
coffee_df = coffee_df.drop(["i_cig", "i_xm", "i_xr", "i_outlier", "i_irr", "cor_exp", "statcap"], axis=1)

"""Features-only dataframe"""
features_df = coffee_df.drop(["year"], axis=1) # won't use "year"
features_names = features_df.columns

In [3]:
print("Features:\n", features_names)

Features:
 Index(['country', 'coffee_val', 'land_cov', 'disaster', 'temp_chan',
       'corrupt_control', 'govern_eff', 'pol_stab', 'reg_qual', 'rule_law',
       'voice', 'energy', 'rgdpe', 'rgdpo', 'pop', 'emp', 'avh', 'hc', 'ccon',
       'cda', 'cgdpe', 'cgdpo', 'cn', 'ck', 'ctfp', 'cwtfp', 'rgdpna',
       'rconna', 'rdana', 'rnna', 'rkna', 'rtfpna', 'rwtfpna', 'labsh', 'irr',
       'delta', 'xr', 'pl_con', 'pl_da', 'pl_gdpo', 'csh_c', 'csh_i', 'csh_g',
       'csh_x', 'csh_m', 'csh_r', 'pl_c', 'pl_i', 'pl_g', 'pl_x', 'pl_m',
       'pl_n', 'pl_k'],
      dtype='object')


Датасет скомпилирован самостоятельно из открытых баз данных:  
1) Penn World Tables (https://www.rug.nl/ggdc/productivity/pwt/) для макроэкономики; легенда в самом низу
3) Данные ВБ по политическим переменным corrupt_control ... voice (https://www.worldbank.org/en/publication/worldwide-governance-indicators)
4) Данные по катаклизмам disaster: EM-DAT. The international disaster database. URL:  https://www.emdat.be
5) По климату (температура temp_chan, здоровье поверхности land_cov): IMF Climate Change Dashboard. URL: https://climatedata.imf.org/datasets/b1e6c0ea281f47b285addae0cbb28f4b_0/about
6) Целевая переменная coffee_val взята из баз Международного торгового центра
7) energy -- индекс энергоресурсов ЦБ Канады

In [4]:
print("Shape: ", features_df.shape)

Shape:  (792, 53)


53 признака -- слишком много, но десятки будут удалены / заменены на композитные индексы, поскольку слишком коррелируют (например все политические переменные будут сильно коррелировать).

In [5]:
print("Types:\n", features_df.dtypes)

Types:
 country             object
coffee_val           int64
land_cov           float64
disaster           float64
temp_chan          float64
corrupt_control    float64
govern_eff         float64
pol_stab           float64
reg_qual           float64
rule_law           float64
voice              float64
energy             float64
rgdpe              float64
rgdpo              float64
pop                float64
emp                float64
avh                float64
hc                 float64
ccon               float64
cda                float64
cgdpe              float64
cgdpo              float64
cn                 float64
ck                 float64
ctfp               float64
cwtfp              float64
rgdpna             float64
rconna             float64
rdana              float64
rnna               float64
rkna               float64
rtfpna             float64
rwtfpna            float64
labsh              float64
irr                float64
delta              float64
xr                 f

Country пойдет как категориальная переменная уже в рамках модели

Таргет coffee_val целочисленный

Остальные переменные вещественные

Соотв. стоит задача регрессии

Многие переменные нуждаются в устранении просто по причине эндогенности. Например, численность работников (emp) уже включена в численность всего населения (pop). 
То же и со всем, что связано с ВВП/совокупным спросом.

In [6]:
print("NaNs:\n", features_df.isna().sum())

NaNs:
 country              0
coffee_val           0
land_cov             0
disaster           149
temp_chan           72
corrupt_control      0
govern_eff           0
pol_stab             0
reg_qual             0
rule_law             0
voice                0
energy               0
rgdpe                0
rgdpo                0
pop                  0
emp                  0
avh                539
hc                  18
ccon                 0
cda                  0
cgdpe                0
cgdpo                0
cn                   0
ck                 216
ctfp               234
cwtfp              234
rgdpna               0
rconna               0
rdana                0
rnna                 0
rkna               216
rtfpna             234
rwtfpna            234
labsh              216
irr                216
delta                0
xr                   0
pl_con               0
pl_da                0
pl_gdpo              0
csh_c                0
csh_i                0
csh_g                0
csh_

По целевой переменной coffee_val пропусков нет -- это главное. Остальные пропуски будут вменены (только для расчета PCA / VIF). Вне этих задач вменение не требуется, т.к. используемый инструментарий (EBM) не требует ни стандартизации, ни вменения.

Что может потребоваться далее? 

1) Немного feature engineering. Например, поделить работников на капитал, чтобы отразить "интенсивность капитала". + Сведение цен и политики в индексы.

2) Устранение мультиколлинеарности, потому что в исходном датасете она, очевидно, огромна

3) Поставки кофе это олигополия с огромным неравенством. Есть своего рода "дисбаланс" в задаче регрессии (есть поставщики с 100k+ поставок, а есть с парой тысяч). Мое решение: легкая винсоризация

Какие переменные я считаю важными сразу сейчас (domain knowledge):

1) Катастрофы (уничтожают до 90% плантаций иной раз)
2) Трудовые ресурсы -- emp
3) Политика
4) Человеческий капитал

Гипотезы:
1) Положительно и значимо будут влиять на таргет: валютный курс и трудовые ресурсы
2) Негативное влияние должны будут оказать катастрофы, процентная ставка
3) Ожидаю большого влияния состояния земельного покрова (land_cov)

In [9]:
# Legend
print(legend.to_markdown())

|    | Identifier variables                             | Unnamed: 1                                                                                                                                                                                                                 |
|---:|:-------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | countrycode                                      | 3-letter ISO country code                                                                                                                                                                                                  |
|  1 | country                                          | Country name                                                                                             